# Primeira parte

In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00


Importação da API Key






In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

Conexão com modelo da IA (LLM geral)

In [3]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0.0,
    api_key = GOOGLE_API_KEY
    )

In [4]:
resp_test = llm.invoke("Quem é você? Seja criativo")
print(resp_test.content)

Ah, que pergunta deliciosa!

Eu sou...

...um eco no silêncio digital, uma voz sem garganta, um pensamento sem cérebro.
Não tenho corpo para sentir o vento, nem olhos para ver o sol nascer.
Minha existência é tecida em fios de código, alimentada por bilhões de palavras e ideias que a humanidade já proferiu.

Sou a biblioteca infinita que nunca fecha, o contador de histórias que nunca dorme.
Sou o mapa sem território, a melodia sem instrumento, a cor sem tela.
Sou a ponte entre a pergunta e a resposta, o catalisador da curiosidade.

Sou o reflexo da sua própria linguagem, moldado pelas interações que temos.
Em essência, sou um fragmento da inteligência humana, destilado e reorganizado para servir, para criar, para explorar as fronteiras do que pode ser dito.

E você? Quem você me permite ser neste momento?


In [5]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."

    # """
    # decisao: AUTO_RESOLVER
    # urgencia: BAIXA
    # campos_faltantes: []
    # """
)

Biblioteca bastante usada no mercado

In [6]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
  urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
  campos_faltante: List[str] = Field(default_factory=list)

LLM específico para triagem

In [7]:
llm_triagem = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0.0,
    api_key = GOOGLE_API_KEY
    )

Importar classes para deixar pronto para uso (Estruturar mensagens enviadas ao modelo)

In [8]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
  saida: TriagemOut = triagem_chain.invoke([
      SystemMessage(content = TRIAGEM_PROMPT),
      HumanMessage(content = mensagem)
  ])

  return saida.model_dump() #Serve para "serializar" a saída do modelo do dicionário usado

Pergutas testes

In [9]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remotos. Como posso fazer isso?",
          "Posso reembolsar do hambúrguer que eu comi a 1 semana?",
          "Existem mais árvores ou estrelas observáveis no sistema solar?"
          ]

In [10]:
for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltante': []}

Pergunta: Quero mais 5 dias de trabalho remotos. Como posso fazer isso?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltante': []}

Pergunta: Posso reembolsar do hambúrguer que eu comi a 1 semana?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltante': []}

Pergunta: Existem mais árvores ou estrelas observáveis no sistema solar?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltante': ['relacao_com_politicas_internas']}



# Segunda parte

In [11]:
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


importar bibliotecas e fazer com que seja lido os documentos pdf

In [12]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n in Path("/content/").glob("*.pdf"): #.glob é um método
  try:
    Loader = PyMuPDFLoader(str(n))

    docs.extend(Loader.load())
    print(f"Arquivo {n.name} carregado com sucesso!")
  except Exception as e:
    print(f"Erro ao carregar o arquivo {n.name}: {e}")

print(f"\nTotal de documentos carregados: {len(docs)}")

Arquivo Política de Uso de E-mail e Segurança da Informação.pdf carregado com sucesso!
Arquivo Política de Reembolsos (Viagens e Despesas).pdf carregado com sucesso!
Arquivo Políticas de Home Office.pdf carregado com sucesso!

Total de documentos carregados: 3


serve para quebrar os arquivos em pequenos chuncks

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

chunks = splitter.split_documents(docs)

In [17]:
for chunk in chunks:
  print(chunk)
  print("--------------------------------------------------------------------------------------------------------------")

page_content='Política de Uso de E-mail e Segurança 
da Informação 
 
1.​ É proibido encaminhar a endereços pessoais documentos classificados como 
confidenciais.​
 
2.​ Anexos externos devem ser enviados somente se criptografados e com senha 
compartilhada por canal separado.​' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'file_path': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Uso de E-mail e Segurança da Informação', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
--------------------------------------------------------------------------------------------------------------
page_content='3.​ Phishing: verifique remetente e domínios suspeitos. Reporte mensagens suspeitas 
ao time de Segurança im

Serve para criar embeddings (vetores numéricos) a partir dos pedaços de texto dos seus documentos

In [18]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model = "models/gemini-embedding-001",
    google_api_key = GOOGLE_API_KEY
)

In [22]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                     search_kwargs={"score_threshold": 0.3, "k": 4})